<h1>Draft Stars</h1>
<p>This program creates an AI model which takes a Brawl Stars match as input, and outputs a number from 0 to 1 predicting which team will win. Values closer to 0 mean the model favors the left, and vice versa.</p>

<h2>Input requirements:</h2>
<p><strong> - CSV of battles:</strong> Check out my repository which collects data and creates the csv: https://github.com/mcmckinley/DraftStarsDataCollection</p>

<h2>Output:</h2>
<p><strong> - The model weights</strong></p>
<p><strong> - An embedding layer for the brawlers</strong></p>
<p><strong> - An embedding layer for the map</strong></p>
<p> - A list of maps in the order than the model knows to interpret them. This must be used whenever inference is preformed on the model</p>


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

<h1>SECTION 1</h1>
<h2>Data - Import, balance, weigh, split</h2>

In [ ]:
# 1.0 Import data

url = 'https://drive.google.com/uc?export=download&id=14oFQeB8Dmv-5nBZKwAVax7qcegRfRz6n' # 750k battles, September 10th

raw_df = pd.read_csv(url)

In [ ]:
# 1.1 Remove battles containing players with too high/low of a skill level. These are hard to interpret.

df = raw_df.copy()

print('Total battles:', len(df))

max_trophies = 1000

columns_to_check = ['a1_t', 'a2_t', 'a3_t', 'b1_t', 'b2_t', 'b3_t']

# Apply the threshold condition to all specified columns
df = df.loc[(raw_df[columns_to_check] < max_trophies).all(axis=1)]

print("Removed battles above", max_trophies, ':', len(df))

min_trophies = 500

df = df.loc[(raw_df[columns_to_check] > min_trophies).all(axis=1)]

print("Removed battles below", min_trophies, ':', len(df))

Total battles: 723763
Removed battles above 1000 : 562431
Removed battles below 500 : 479776


In [ ]:
# These maps contain bad data
df = df[df['map'] != 'Rusty Rebound']
df = df[df['map'] != 'Spring Back Alley']
df = df[df['map'] != 'Touch Up Tavern']
df = df[df['map'] != 'Greasepaint Grass']
df = df[df['map'] != 'Dye Direct']
df = df[df['map'] != 'Chromatic Cress']
df = df[df['map'] != 'Spots Of Yore']
df = df[df['map'] != 'Tint Terrace']
df = df[df['map'] != 'Shooting Star']
df = df[df['map'] != 'Layer Cake']
df = df[df['map'] != 'Hideout']
df = df[df['map'] != 'Temple Ruins']
df = df[df['map'] != 'Split']
df = df[df['map'] != 'Canal Grande']
df = df[df['map'] != 'Galaxy Arena']
df = df[df['map'] != 'Excel']
df = df[df['map'] != 'Outlaw Camp']
df = df[df['map'] != 'Triple Dribble']
df = df[df['map'] != 'Bandit Stash']
df = df[df['map'] != 'Pit Stop']
df = df[df['map'] != 'Bridge Too Far']
df = df[df['map'] != 'Crystal Arcade']
print("Length ithout these maps:", len(df))

Length ithout these maps: 426319


In [ ]:
# 1.2 - Balance data

# It is crucial that our data has an equal number of victories / defeats on either side,
# so that the model doesn't learn to prefer one side over the other.
# To solve this, we flip some of the matches so that the winner is on the other side.

# Count the occurrences of 1s and 0s
counts = df['did_blue_team_win'].value_counts()
team_a_victories = counts.get(0, 0)
team_b_victories = counts.get(1, 0)
print("Number of times blue won:", team_a_victories)
print("Number of times red won:", team_b_victories)

num_rows_to_flip = (team_a_victories - team_b_victories) // 2

overrepresented_bit = (0 if num_rows_to_flip < 0 else 1)

num_rows_to_flip = abs(num_rows_to_flip)

print('Will flip', num_rows_to_flip, 'rows')

# Identify which rows should be candidates to be flipped
rows_to_flip = df['did_blue_team_win'] == overrepresented_bit

# Get the indices of the rows to flip
indices_to_flip = df[rows_to_flip].index[:num_rows_to_flip]

# Perform the flipping operations on the selected rows
# Flip 'did_blue_team_win' values
df.loc[indices_to_flip, 'did_blue_team_win'] = 1 - df.loc[indices_to_flip, 'did_blue_team_win']

# Flip the brawler IDs and trophy counts
for x in range(1, 4):
    a_col = 'a' + str(x)
    b_col = 'b' + str(x)
    a_t_col = a_col + '_t'
    b_t_col = b_col + '_t'

    # Use temporary variable to swap the columns
    temp = df.loc[indices_to_flip, a_col].copy()
    df.loc[indices_to_flip, a_col] = df.loc[indices_to_flip, b_col]
    df.loc[indices_to_flip, b_col] = temp

    temp = df.loc[indices_to_flip, a_t_col].copy()
    df.loc[indices_to_flip, a_t_col] = df.loc[indices_to_flip, b_t_col]
    df.loc[indices_to_flip, b_t_col] = temp

print('Data has been balanced.')

counts = df['did_blue_team_win'].value_counts()
team_a_victories = counts.get(0, 0)
team_b_victories = counts.get(1, 0)
print("Number of times blue won:", team_a_victories)
print("Number of times red won:", team_b_victories)

Number of times blue won: 212467
Number of times red won: 213852
Will flip 693 rows
Data has been balanced.
Number of times blue won: 213160
Number of times red won: 213159


In [ ]:
# 1.3 - Weigh each match based on skill:
#       - If a high trophy team wins, this isn't too meaningful.
#       - If a low trophy team wins, we assume it's because they have a superior team composition.
#         Thus we tell the model to consider these matches more heavily.

df['blue_team_highest_trophies'] = df[['a1_t', 'a2_t', 'a3_t']].max(axis=1)
df['red_team_highest_trophies'] = df[['b1_t', 'b2_t', 'b3_t']].max(axis=1)

df['blue_team_trophy_advantage'] = df['blue_team_highest_trophies'] - df['red_team_highest_trophies']

# Calculate weight based on:
#   - who has the trophy advantage
#   - who won
e = 2.71
k = 0.01
df['weight'] = (np.where(df['did_blue_team_win'] == 1,
                         e**(-k * (df['blue_team_trophy_advantage'])),   # if team B wins (notice the -k)
                         e**( k * (df['blue_team_trophy_advantage']))))  # if team A wins

In [ ]:
# 1.4 - Drop data that is no longer necessary

columns_to_drop = ['a1_t', 'a2_t', 'a3_t', 'b1_t', 'b2_t', 'b3_t', 'blue_team_highest_trophies', 'red_team_highest_trophies', 'blue_team_trophy_advantage']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
# 1.5 - Convert the data into integer arrays

brawlers = ['SHELLY', 'COLT', 'BULL', 'BROCK', 'RICO', 'SPIKE', 'BARLEY', 'JESSIE', 'NITA', 'DYNAMIKE', 'EL PRIMO', 'MORTIS', 'CROW', 'POCO', 'BO', 'PIPER', 'PAM', 'TARA', 'DARRYL', 'PENNY', 'FRANK', 'GENE', 'TICK', 'LEON', 'ROSA', 'CARL', 'BIBI', '8-BIT', 'SANDY', 'BEA', 'EMZ', 'MR. P', 'MAX', 'empty1', 'JACKY', 'GALE', 'NANI', 'SPROUT', 'SURGE', 'COLETTE', 'AMBER', 'LOU', 'BYRON', 'EDGAR', 'RUFFS', 'STU', 'BELLE', 'SQUEAK', 'GROM', 'BUZZ', 'GRIFF', 'ASH', 'MEG', 'LOLA', 'FANG', 'empty2', 'EVE', 'JANET', 'BONNIE', 'OTIS', 'SAM', 'GUS', 'BUSTER', 'CHESTER', 'GRAY', 'MANDY', 'R-T', 'WILLOW', 'MAISIE', 'HANK', 'CORDELIUS', 'DOUG', 'PEARL', 'CHUCK', 'CHARLIE', 'MICO', 'KIT', 'LARRY & LAWRIE', 'MELODIE', 'ANGELO', 'DRACO', 'LILY', 'BERRY', 'CLANCY']
maps = list(set(df['map'].values)) # Take the maps column and remove duplicates

num_brawlers = len(brawlers)
num_maps = len(maps)

print(f'number of brawlers: {num_brawlers}')
print(f'number of maps: {num_maps}')

def brawler_index(brawler):
    return brawlers.index(brawler)

def map_index(map):
    return maps.index(map)

df['map_index'] = df['map'].apply(map_index)

data_list = df[['a1', 'a2', 'a3', 'b1', 'b2', 'b3', 'map_index', 'did_blue_team_win']]
data_list_weights = df['weight']
data_list = data_list.to_numpy()

print(data_list[0]) # to show that the data is being interpreted correctly

number of brawlers: 84
number of maps: 52
[15  3 45 83  1 15  8  0]


In [ ]:
# 1.7 - Convert dataset into train_test_split

from sklearn.model_selection import train_test_split

input_data = df[['a1', 'a2', 'a3', 'b1', 'b2', 'b3', 'map_index']]
target_data = df['did_blue_team_win']
weight_data = df['weight']

input_data = input_data.to_numpy()
target_data = target_data.to_numpy()
weight_data = weight_data.to_numpy()

print(type(input_data))
print(input_data.shape)

train_input, test_input, train_target, test_target, train_weight, test_weight = train_test_split(input_data, target_data, weight_data, test_size=0.05, random_state=42)
train_input = torch.tensor(train_input, dtype=torch.int)
test_input = torch.tensor(test_input, dtype=torch.int)
train_target = torch.tensor(train_target, dtype=torch.float)
test_target = torch.tensor(test_target, dtype=torch.float)
train_weight = torch.tensor(train_weight, dtype=torch.float)
test_weight = torch.tensor(test_weight, dtype=torch.float)

<class 'numpy.ndarray'>
(482349, 7)


In [ ]:
# 1.8 - Use loading system on dataset
batch_size = 1024

from torch.utils.data import Dataset, DataLoader

class WeightedDataset(Dataset):
    def __init__(self, data, targets, weights):
        self.data = data
        self.targets = targets
        self.weights = weights

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx], self.weights[idx]

train_dataset = WeightedDataset(train_input, train_target, train_weight)
test_dataset = WeightedDataset(test_input, test_target, test_weight)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

<h1>SECTION 2</h1>
<h2>Import manually predetermined weights</h2>

In [ ]:
# 2.0 - Initialize brawler weights with values that have been manually determined (by me)

# This is one thing that differs my model from ChatGPT.
# I have the luxury of manually initializing my values, based on real traits that each character has.

brawler_data_spreadsheet = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRZXNyNjU1csRxyikhZ-GbnLrt-99bX0FCvxnBKzobXtXpWmvJl8gtNfb46CDIZ50LLHWoY9JU-U4A2/pub?output=csv'
brawler_data_df = pd.read_csv(brawler_data_spreadsheet)

# Drop these columns
columns_to_drop = ['mico', 'chuck', 'mortis', 'sprout', 'bonnie', 'draco']
for column in columns_to_drop:
  brawler_data_df.drop(column, axis=1, inplace=True)



brawler_data_list = brawler_data_df.to_numpy()
traits_per_brawler = len(brawler_data_list[0]) - 1 # minus 1 for the name column

# First, set the brawler embeddings to 0
initial_brawler_embedding = torch.zeros(num_brawlers, traits_per_brawler)

# Then, update the embedding weights
for brawler_data in brawler_data_list:
  numeric_values = [float(val) for val in brawler_data[1:]]
  initial_brawler_embedding[brawler_index(brawler_data[0])] = torch.tensor(numeric_values)

# Add blank columns for the model to use freely
num_extra_columns_to_add = 10
print(f'using {num_extra_columns_to_add} blank columns')
extra_columns = torch.zeros(num_brawlers, num_extra_columns_to_add)
initial_brawler_embedding = torch.cat([initial_brawler_embedding, extra_columns], dim=1)

initial_brawler_embedding = nn.Embedding.from_pretrained(initial_brawler_embedding, freeze=False)

traits_per_brawler += num_extra_columns_to_add
print('Traits per brawler:', traits_per_brawler)
print(initial_brawler_embedding)

using 10 blank columns
Traits per brawler: 44
Embedding(84, 44)


In [ ]:
# 2.0b - Initialize map weights with manually determined values

url_of_map_data = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRnV3wZG4MMDm2mROwP8ejxaolBuYuV7BjygLlA2X6FLsLmYp3zplalDfveFTJPKZVpcCPWxCSJr4Y3/pub?output=csv'
map_data_df = pd.read_csv(url_of_map_data)
map_data_list = map_data_df.to_numpy()

traits_per_map = len(map_data_list[0]) - 1 # minus 1 for the map's name

# Initialize the map embeddings to 0
initial_map_embedding = torch.zeros(num_maps, traits_per_map)

# Set the map embedding weights
for map_data in map_data_list:
  numeric_values = [float(val) for val in map_data[1:]]
  try:
    initial_map_embedding[map_index(map_data[0])] = torch.tensor(numeric_values)
  except ValueError: # A given map might not necessarily occur in the dataset.
    continue

initial_map_embedding = nn.Embedding.from_pretrained(initial_map_embedding, freeze=False)

print('Traits per map:', traits_per_map)
print(initial_map_embedding)

Traits per map: 10
Embedding(52, 10)


In [ ]:
# 2.1 - Identify if any maps don't have pretrained embeddings
for map in maps:
  is_in_array = False
  for other_map in map_data_list:
    if map == other_map[0]:
      is_in_array = True
      break
  if not is_in_array:
    print(map)

<h1>SECTION 3</h1>
<h2>Define & Initialize the model</h2>

In [ ]:
# 3.0 - Define the model

import torch.nn as nn

class Model(nn.Module):
    def __init__(self, brawler_emb_dim, initial_brawler_embedding, map_emb_dim, initial_map_embedding, num_heads, num_layers, dim_feedforward):
        super(Model, self).__init__()
        self.brawler_embedding = initial_brawler_embedding
        self.map_embedding = initial_map_embedding

        encoder_layer = nn.TransformerEncoderLayer(
            d_model = brawler_emb_dim + map_emb_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            batch_first=True  # Set batch_first to True
        )

        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear((brawler_emb_dim + map_emb_dim) * 6, 1)  # Assuming 3 characters per match
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        brawlers, map = torch.split(x, [6, 1], dim=1)
        brawler_embedding = self.brawler_embedding(brawlers)

        # Add the map on top of each brawler embedding. Thus, each brawler has their own 'impression' of the map.
        map_embedding = self.map_embedding(map).repeat(1, 6, 1)

        embeddings = torch.cat([map_embedding, brawler_embedding], dim=2)

        transformer_output = self.transformer_encoder(embeddings)

        # Flatten the output into a 1D array
        flattened_output = transformer_output.reshape(transformer_output.size(0), -1)

        # Pass the flattened output through a fully connected layer
        output = self.fc(flattened_output)

        return self.sigmoid(output)

In [ ]:
# 3.1 - Hyperparameters

brawler_vocab_size = num_brawlers
map_vocab_size = num_maps

brawler_emb_dim = traits_per_brawler
map_emb_dim = traits_per_map

# we can manually run more later
num_epochs = 1

num_heads = 2
num_layers = 2
dim_feedforward = 64  # Dimension of the last feedforward network model

In [ ]:
# 3.2 - Test that the model works

model = Model(brawler_emb_dim, initial_brawler_embedding, map_emb_dim, initial_map_embedding, num_heads, num_layers, dim_feedforward)
test_input = torch.tensor([[0,2,3,81,5,6,7]])
output = model(test_input)
print(output)

torch.Size([1, 6, 54])
torch.Size([1, 6, 54])
tensor([[0.4609]], grad_fn=<SigmoidBackward0>)


In [ ]:
# 3.3 - Initialize the model, loss function and optimizer

torch.manual_seed(41)

# Model initialization
model = Model(brawler_emb_dim, initial_brawler_embedding, map_emb_dim, initial_map_embedding, num_heads, num_layers, dim_feedforward)

# Binary Cross-Entropy loss for binary classification - what we're doing is predicting who wins
criterion = nn.BCELoss(reduction='none')

optimizer = optim.Adam([
    {'params': model.brawler_embedding.parameters(), 'lr': 0.015},
    {'params': model.map_embedding.parameters(), 'lr': 0.015},
    {'params': model.fc1.parameters(), 'lr': 0.001},
    {'params': model.fc2.parameters(), 'lr': 0.001},
    {'params': model.fc3.parameters(), 'lr': 0.001},
    {'params': model.fc4.parameters(), 'lr': 0.001},
    {'params': model.sigmoid.parameters(), 'lr': 0.0001},
])

<h1>SECTION 4</h1>
<h2>Testing the Model</h2>


In [ ]:
# 4.1 - Initialize

current_epoch = 0
try:
    current_epoch = checkpoint['epoch']
except NameError:
    current_epoch = 0
except KeyError:
    current_epoch = 0

epoch_losses = []

full_dataset_predictions_per_epoch = [] # on all matches
partial_dataset_predictions_per_epoch = [] # on matches with a weight greater than 1

In [ ]:
# 4.2 - To run more epochs on the model, run this box and everything below it

epoch_to_end_on = current_epoch + num_epochs
print(f'Current epoch: {current_epoch}')
print(f'Will run {num_epochs} epochs')
print(f'Stopping at epoch {epoch_to_end_on}')

Current epoch: 1
Will run 1 epochs
Stopping at epoch 2


In [ ]:
# 4.3 - Training loop
model.train()
while current_epoch < epoch_to_end_on:
    epoch_loss = 0
    current_epoch += 1

    i = 0
    for batch in train_dataloader:
        inputs, targets, weights = batch

        optimizer.zero_grad()
        pred = model(inputs)
        targets = targets.unsqueeze(1)
        loss = criterion(pred, targets)
        weighted_loss = (loss * weights).mean()
        weighted_loss.backward()
        optimizer.step()

        epoch_loss += weighted_loss.item()
        #if (i % 25 == 0):
        #    print(f'Batch {i} loss: {weighted_loss.item()}')
        i += 1

    print(f'Epoch {current_epoch}: Loss = {weighted_loss}')
    epoch_losses.append(weighted_loss)

Epoch 2: Loss = 0.6661221981048584


In [ ]:
# 4.4 - Show gradients
#for name, param in model.named_parameters():
#    if param.grad is not None:
#        print(f'Gradient for {name}: {param.grad.norm()}')

In [ ]:
for i in range(len(epoch_losses)):
  print(f'Epoch {i}: Loss = {epoch_losses[i]}')

Epoch 0: Loss = 0.6997237205505371
Epoch 1: Loss = 0.6661221981048584


<h1>SECTION 5</h1>
<h2>Evaluating the Model</h2>

In [ ]:
# 5.0 - Evaluate the model

# Evaluation loop
model.eval()
test_loss = 0
predictions, true_labels, weight_labels = [], [], []

with torch.no_grad():
    for batch in test_dataloader:
        inputs, targets, weights = batch
        outputs = model(inputs).squeeze()
        targets = targets.squeeze()

        loss = criterion(outputs, targets)
        weighted_loss = loss * weights
        test_loss += weighted_loss.mean().item()

        # Collect predictions and true labels for display
        predictions.extend(outputs.cpu().numpy())
        true_labels.extend(targets.cpu().numpy())
        weight_labels.extend(weights.cpu().numpy())

# Display a few predictions and their corresponding true labels

num_right, total, f_num_right, f_total = 0, 0, 0, 0

for predicted_val, target_val, weight in zip(predictions, true_labels, weight_labels):
    correct = (predicted_val > 0.5) == (target_val == 1.0)
    f_total += 1
    if correct:
      f_num_right += 1
    if weight > 1:
      total += 1
      if correct:
        num_right += 1


full_dataset_predictions_per_epoch.append(f_num_right / f_total)

partial_dataset_predictions_per_epoch.append(num_right / total)

print('Prediction success rate on full training dataset')
for prediction in full_dataset_predictions_per_epoch:
  print(round(prediction, 3))

print('Prediction success rate on training dataset where the winning team has lower trophies')
for prediction in partial_dataset_predictions_per_epoch:
  print(round(prediction, 3))

print()
print('Example predictions')
for i in range(10):
    print(f'pred: {predictions[i]:.2f}, target: {true_labels[i]}')

Predictions on full training dataset
0.572
0.576
Predictions on training dataset where the winning team has lower trophies
0.588
0.593

Example predictions
pred: 0.56, target: 1.0
pred: 0.20, target: 0.0
pred: 0.70, target: 1.0
pred: 0.49, target: 0.0
pred: 0.56, target: 1.0
pred: 0.65, target: 1.0
pred: 0.54, target: 1.0
pred: 0.60, target: 1.0
pred: 0.52, target: 0.0
pred: 0.43, target: 1.0


In [ ]:
# 5.1 - Example final brawler recommendation

def recommend_brawler(battle):
    model.eval()
    guesses = []
    # Convert match elements to indices
    for i in range(6):
        try:
          battle[i] = brawler_index(battle[i])
        except ValueError:
          continue

    battle[6] = map_index(battle[6])

    guesses = []

    for i in range(len(brawlers)):
        battle[5] = i  # Set the current brawler index in match

        input_tensor = torch.tensor(battle, dtype=torch.long).unsqueeze(0)
        output = model(input_tensor).squeeze()

        output_value = round(float(output.item()), 2)

        guess = {
            'id': int(i),
            'val': output_value
        }
        guesses.append(guess)


    guesses = sorted(guesses, key=lambda x: x['val'], reverse=False)
    for guess in guesses[:10]:
        print(f'{brawlers[guess["id"]]} -', end=' ')



print('Infinite Doom - Frank 5x:')
recommend_brawler(['FRANK', 'FRANK', 'FRANK', 'FRANK', 'FRANK', '?', 'Infinite Doom']) # should recommend a tank buster

print('\n\nDouble Swoosh: SANDY LEON POCO:')
recommend_brawler(['SANDY', 'LEON', 'POCO', 'SPIKE', 'JESSIE', '?', "Double Swoosh"])  # should recommend crow, maybe gene

print('\n\nHot Potato: BIBI CHUCK BULL:')
recommend_brawler(['BIBI', 'CHUCK', 'BULL', 'COLT', 'DYNAMIKE', '?', 'Hot Potato'])   # should recommend a good heist brawler;

print('\n\nHot Potato: BIBI COLT DYNAMIKE')
recommend_brawler(['BIBI', 'COLT', 'DYNAMIKE', 'BARLEY', 'EDGAR', '?', 'Hot Potato'])   # should recommend a good heist brawler;

print('\n\nHot Potato: MELODIE COLT DYNAMIKE')
recommend_brawler(['MELODIE', 'COLT', 'DYNAMIKE', 'BULL', 'ROSA', '?', 'Hot Potato'])   # should recommend a good heist brawler;

print('\n\nDeep End: NANI COLT EVE')
recommend_brawler(['NANI', 'COLT', 'EVE', 'BROCK', '8-BIT', '?', 'Deep End'])         # sniper

print('\n\nHot Zone: JACKY BULL HANK:')
recommend_brawler(['JACKY', 'BULL', 'HANK', 'BYRON', 'POCO', '?', 'Ring of Fire'])    # resistance


Infinite Doom - Frank 5x:
LOLA - CLANCY - AMBER - EVE - JANET - OTIS - R-T - DRACO - BARLEY - PAM - 

Double Swoosh: SANDY LEON POCO:
LOLA - CLANCY - AMBER - EVE - JANET - OTIS - R-T - BARLEY - 8-BIT - RUFFS - 

Hot Potato: BIBI CHUCK BULL:
LOLA - CLANCY - JANET - OTIS - R-T - BARLEY - 8-BIT - AMBER - RUFFS - EVE - 

Hot Potato: BIBI COLT DYNAMIKE
LOLA - CLANCY - JANET - R-T - BARLEY - 8-BIT - AMBER - RUFFS - EVE - OTIS - 

Hot Potato: MELODIE COLT DYNAMIKE
LOLA - CLANCY - R-T - BARLEY - AMBER - RUFFS - EVE - JANET - OTIS - DRACO - 

Deep End: NANI COLT EVE
LOLA - CLANCY - R-T - AMBER - EVE - JANET - OTIS - DRACO - BARLEY - 8-BIT - 

Hot Zone: JACKY BULL HANK:
LOLA - CLANCY - JANET - OTIS - R-T - BARLEY - 8-BIT - AMBER - RUFFS - EVE - 

In [ ]:
# 5.2 - Test the model's consistency.
#       We want a model that is permutation invariant - meaning it should give the same results regardless of how a team is ordered.
#       I haven't come up with a solution for this yet.

same_battles = [
    [0, 1, 2, 3, 4, 5, 6],
    [0, 2, 1, 3, 5, 4, 6],
    [1, 2, 0, 4, 5, 3, 6],
    [1, 0, 2, 4, 3, 5, 6],
    [2, 0, 1, 5, 3, 4, 6],
    [2, 1, 0, 5, 4, 3, 6]
]

print('These battles should produce near identical outputs:')
for battle in same_battles:
  input = torch.tensor(battle).unsqueeze(0)
  output = model(input)
  print(round(output.squeeze().item(), 5))

mirror_matchups = [
    [0, 0, 0, 0, 0, 0, 6],
    [1, 1, 1, 1, 1, 1, 6],
    [2, 2, 2, 2, 2, 2, 6],
    [3, 3, 3, 3, 3, 3, 6],
    [4, 4, 4, 4, 4, 4, 6],
    [5, 5, 5, 5, 5, 5, 6],
    [6, 6, 6, 6, 6, 6, 6]
]

print('Mirror matchups, should be close to 0.5:')
for battle in mirror_matchups:
  input = torch.tensor(battle).unsqueeze(0)
  output = model(input)
  print(round(output.squeeze().item(), 5))

<h1>SECTION 6</h1>
<h2>Saving the Model</h2>

In [ ]:
# 6.1 - Save the model for inference
#torch.save(model.state_dict(), 'bm_7_4_611-612_withgaps.pt')

In [ ]:
# 6.2 - Saving the model for training
#torch.save({
#    'epoch': current_epoch,
#    'model_state_dict': model.state_dict(),
#    'optimizer_state_dict': optimizer.state_dict(),
#    'loss': epoch_losses[-1],  # Save the final epoch's average loss
#}, 'brawl_mind_6_25.pt')

In [ ]:
# 6.3 - Identify if any maps don't have pretrained embeddings
for map in maps:
  is_in_array = False
  for other_map in map_data_list:
    if map == other_map[0]:
      is_in_array = True
      break
  if not is_in_array:
    print(map)

In [ ]:
# 6.4
# ! IMPORTANT !
# Add this into the backend's env file
# We need to know the order the maps are in
for map in maps:
  print('"' + map + '",', end=' ')


# Currently, the model has no training data on these maps, but we can initialize them manually the same way we do with all the other maps.
# Just means that the model can't adjust them.
print(' "Canal Grande", "Hideout", "Shooting Star",')

In [ ]:
# 6.5 - Save the model and embeddings
#torch.save(model.state_dict(), 'draftstars.pt')
#torch.save(model.brawler_embedding.weight.detach(), 'brawler_embeddings_V7_6.pt')
#torch.save(model.map_embedding.weight.detach(), 'map_embeddings_V7_6.pt')

In [ ]:
# 6.6 - Download
#from google.colab import files
#files.download('BMV7_6.pt')
#files.download('brawler_embeddings_V7_6.pt')
#files.download('map_embeddings_V7_6.pt')